# Update Countries

In [1]:
import ipywidgets as widgets
from IPython.display import display


In [ ]:
from giga.app.config import get_registered_countries

option = widgets.RadioButtons(
    options=['Update an existing country', 'Add a new country'],
    description='Country:',
    disabled=False,
    index=None
)

country_select = widgets.Dropdown(
    options=get_registered_countries(),
    description='Country:',
)

country_input = widgets.Text(
    description='Country:',
)

output = widgets.Output()

# Create a function to handle changes in the selected option
def on_change(change):
    output.clear_output()
    if change['new'] == 'Update an existing country':
        with output:
            display(country_select)
    else:
        with output:
            display(country_input)

# Set the function to be called when the selected option changes
option.observe(on_change, 'value')

# Display the option
display(option, output)

def creating_new_country():
    return option.value == 'Add a new country'

def get_selected_country():
    if not creating_new_country():
        if country_select.value is not None:
            return country_select.value
    elif country_input.value.strip() != "":
        return country_input.value.strip().replace(" ", "_").lower()
    return None


In [ ]:
from giga.data.store.stores import COUNTRY_DATA_STORE as data_store
from giga.utils.globals import COUNTRY_DEFAULT_RELATIVE_DIR

display("Files required for a new country:")

def up_btn(description, accept=".csv", **kwargs):
    layout = widgets.Layout(width='300px')
    return widgets.FileUpload(accept=accept, multiple=False, 
                              description=description, layout=layout, 
                              max_file_size=100,  # MB
                              **kwargs)

country_defaults = up_btn(accept='.json', description="Country defaults (.json)")
display(country_defaults)

schools = up_btn(accept='.csv', description="Schools (.csv)")
display(schools)

display("Optional files:")

cellular = up_btn(accept='.csv', description="Cell tower locations (.csv)")
display(cellular)

fiber = up_btn(accept='.csv', description="Fiber node locations (.csv)")
display(fiber)

electricity = up_btn(accept='.csv', description="Electricity info (.csv)")
display(electricity)

schools_supplemental = up_btn(accept='.csv', description="Supplemental school info (.csv)")
display(schools_supplemental)

upload_output = widgets.Output()
upload_button = widgets.Button(description="Upload")
display(upload_button, upload_output)

def on_upload_change(change):
    upload_output.clear_output()
    with upload_output:
        country = get_selected_country()
        if country is None:
            print("Must select a country!")
            return
        if creating_new_country() and (not country_defaults.value or not schools.value):
            print("Missing required file!")
            return
        file_paths = {
            country_defaults: f"{COUNTRY_DEFAULT_RELATIVE_DIR}/{country}.json",
            schools: f"/workspace/{country}/schools.csv",
            cellular: f"/workspace/{country}/cellular.csv",
            fiber: f"/workspace/{country}/fiber.csv",
            electricity: f"/workspace/{country}/electricity.csv",
            schools_supplemental: f"/workspace/{country}/schools_supplemental.csv"
        }
        n_updated = 0
        for btn, path in file_paths.items():
            if len(btn.value) == 0:
                if data_store.file_exists(path):
                    continue
                print(f"Writing empty file for {btn.description}, as one does not exist")
                data_store.write_file(path, "")
                continue
            file_info = btn.value[0]
            content = file_info['content'].tobytes().decode()
            print(f"Updated {btn.description}")
            data_store.write_file(path, content)
            n_updated += 1
        print(f"Updated configuration for {country}")
        print(f"Updated {n_updated} file(s).")

upload_button.on_click(on_upload_change)
